reshaping(重构)=转换表或者向量(**DataFrame/Series**)的结构，让其适合进行数据分析

* pivot
* pivot-table
* stack
* un-stack

# pivot(Excel常用，也就是透视图的意思)
pivot函数有三个参数
* index
* columns
* values

In [1]:
from collections import OrderedDict
from pandas import DataFrame
import pandas as pd
import numpy as np

In [2]:
# 下面使用的是有序的字典，里面的顺序是固定的
table = OrderedDict((
    ('Item', ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType', ['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD', [1, 2, 3, 4]),
    ('EU', [1, 2, 3, 4])
))
df = DataFrame(table)
df

,Item,CType,USD,EU
0,Item0,Gold,1,1
1,Item0,Bronze,2,2
2,Item1,Gold,3,3
3,Item1,Silver,4,4


In [4]:
p = df.pivot(index='Item', columns='CType', values='USD')
p    # p里面没有EU信息，某种意义上pivot后是对原来信息的简化。

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,1.0,NaN
Item1,NaN,3.0,4.0


In [6]:
# 原来的DataFrame:获取Item0，金牌客户的价值
print(df[(df.Item=='Item0') & (df.CType=='Gold')].USD.values)

[1]


In [7]:
# 现在的Pivot DataFrame
print(p[p.index=='Item0'].Gold.values)

[ 1.]


# 多列pivot

In [8]:
p = df.pivot(index='Item', columns='CType')
p     # muliindex = hierarchical column

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0    2.0  1.0    NaN    2.0  1.0    NaN
Item1    NaN  3.0    4.0    NaN  3.0    4.0

In [9]:
print(df[(df.Item=='Item0') & (df.CType=='Gold')].USD.values)

[1]


In [10]:
print(p.USD[p.USD.index=='Item0'].Gold.values)

[ 1.]


## pivot常见错误-index/columns索引后对应后有多行相同值
**`这个问题的解决方法是用pivot-table!`**

In [12]:
table = OrderedDict((
    ('Item', ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType', ['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD', [1, 2, 3, 4]),
    ('EU', [1, 2, 3, 4])
))
df = DataFrame(table)
print(df)
p = df.pivot(index='Item', columns='CType', values='USD')

    Item   CType  USD  EU
0  Item0    Gold    1   1
1  Item0  Bronze    2   2
2  Item0    Gold    3   3
3  Item1  Silver    4   4


ValueError: Index contains duplicate entries, cannot reshape

## pivot_table
> **`pivot_table`**解决这个问题的方式是取了平均值

In [14]:
table = OrderedDict((
    ('Item', ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType', ['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD', [1, 2, 3, 4]),
    ('EU', [1, 2, 3, 4])
))
df = DataFrame(table)
p = df.pivot_table(index='Item', columns='CType', values='USD')
p

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,2.0,NaN
Item1,NaN,NaN,4.0


In [15]:
df

,Item,CType,USD,EU
0,Item0,Gold,1,1
1,Item0,Bronze,2,2
2,Item0,Gold,3,3
3,Item1,Silver,4,4


> 在`pivot_table`的参数中，可以通过使用**`aggfunc`** 来指定重复时要采取的取值的方式！

In [16]:
table = OrderedDict((
    ('Item', ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType', ['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD', [1, 2, 3, 4]),
    ('EU', [1, 2, 3, 4])
))
df = DataFrame(table)
p = df.pivot_table(index='Item', columns='CType', values='USD', aggfunc=np.max)    # aggfunc指明aggretation函数
p

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,3.0,NaN
Item1,NaN,NaN,4.0


### stack/unstack
pivot实际是对DataFrame进行stack的一种特例。stack意味把最内层的列索引变成最内层的行索引，而unstack就是把最内层的行索引变成列索引。
![stack_unstack.png](./img/stack_unstack.png)

In [3]:
# Row Multi-Index
row_idx_arr = list(zip(['r0', 'r0'], ['r00', 'r01']))
row_idx = pd.MultiIndex.from_tuples(row_idx_arr)
print(row_idx)
# Column Multi-Index
col_idx_arr = list(zip(['c0','c0', 'c1'], ['c00', 'c01', 'c10']))
col_idx = pd.MultiIndex.from_tuples(col_idx_arr)
print(col_idx)
# Create the DataFrame
df = DataFrame(np.arange(6).reshape(2, 3), index=row_idx, columns=col_idx)
df = df.applymap(lambda x: (x // 3, x % 3))
df

MultiIndex(levels=[['r0'], ['r00', 'r01']],
           labels=[[0, 0], [0, 1]])
MultiIndex(levels=[['c0', 'c1'], ['c00', 'c01', 'c10']],
           labels=[[0, 0, 1], [0, 1, 2]])


c0              c1
           c00     c01     c10
r0 r00  (0, 0)  (0, 1)  (0, 2)
   r01  (1, 0)  (1, 1)  (1, 2)

In [5]:
p = df.stack()
p

c0      c1
r0 r00 c00  (0, 0)     NaN
       c01  (0, 1)     NaN
       c10     NaN  (0, 2)
   r01 c00  (1, 0)     NaN
       c01  (1, 1)     NaN
       c10     NaN  (1, 2)

In [6]:
p.unstack()

c0                c1             
           c00     c01  c10  c00  c01     c10
r0 r00  (0, 0)  (0, 1)  NaN  NaN  NaN  (0, 2)
   r01  (1, 0)  (1, 1)  NaN  NaN  NaN  (1, 2)

## pivot案例

In [7]:
df = pd.read_excel('./data/sales-funnel.xlsx')
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [12]:
df.dtypes

Account        int64
Name          object
Rep           object
Manager       object
Product       object
Quantity       int64
Price          int64
Status      category
dtype: object

In [17]:
# 设置成category是因为本来也就是只有这几样
df['Status'] = df['Status'].astype('category')
# 下面是设置分类变量的顺序
df['Status'].cat.set_categories(['won', 'pending', 'presented', 'declined'],inplace=True)

In [9]:
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [10]:
pd.pivot_table(df, index=['Manager', 'Rep'], values=['Price'])

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

In [18]:
pd.pivot_table(df, index=['Manager', 'Rep'], values=['Price'], aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [19]:
pd.pivot_table(df, index=['Manager', 'Rep'], values=['Price'], aggfunc=[np.mean, len])

mean   len
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4

In [24]:
pd.pivot_table(df, index=['Manager', 'Rep'], columns=['Product'], values=['Price'], aggfunc=[np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

> 像上面这种，一些类型数据没有，显示的是NaN，而我们一般的要求实际上是0，这个时候可以使用 **`fill_value`** 这个参数，并且赋值为 **`0`** 就行！

In [25]:
pd.pivot_table(df, index=['Manager', 'Rep'], columns=['Product'], values=['Price'], aggfunc=[np.sum], fill_value=0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [26]:
# 下面这种，除了统计Price还统计了Quantity！
pd.pivot_table(df, index=['Manager', 'Rep'], columns=['Product'], values=['Price', 'Quantity'], aggfunc=[np.sum], fill_value=0)

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

In [28]:
# 下面这个是把product放到了列的位置！！
pd.pivot_table(df,  index=['Manager', 'Rep', 'Product'], values=['Price', 'Quantity'], aggfunc=[np.sum], fill_value=0)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

> 通过使用 **`margins`** 这个参数，并且赋值为 **`True`** 来在生成的数据的最下面生成数据的总和。

In [32]:
pd.pivot_table(df,  index=['Manager', 'Rep', 'Product'], 
               values=['Price', 'Quantity'], aggfunc=[np.sum, np.mean], fill_value=0, margins=True)

sum            mean          
                                          Price Quantity  Price  Quantity
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500  1.000000
                            Maintenance    5000        2   5000  2.000000
                            Software      10000        1  10000  1.000000
              Daniel Hilton CPU          105000        4  52500  2.000000
                            Software      10000        1  10000  1.000000
              John Smith    CPU           35000        1  35000  1.000000
                            Maintenance    5000        2   5000  2.000000
Fred Anderson Cedric Moss   CPU           95000        3  47500  1.500000
                            Maintenance    5000        1   5000  1.000000
                            Software      10000        1  10000  1.000000
              Wendy Yule    CPU          165000        7  82500  3.500000
                            Maintenance    7000        3   7000  3.000000
                            Monitor        5000        2   5000  2.000000
All                                      522000       30  30705  1.764706

In [33]:
pd.pivot_table(df, index=['Manager', 'Status'], values=['Price'], 
              aggfunc=[np.sum], fill_value=0, margins=True)

sum
                          Price
Manager       Status           
Debra Henley  won         65000
              pending     50000
              presented   50000
              declined    70000
Fred Anderson won        172000
              pending      5000
              presented   45000
              declined    65000
All                      522000

In [41]:
# 下面通过传字典来改变列名。 这个实际上显示更加简洁了 ！！！！
pd.pivot_table(df, index=['Manager', 'Status'], columns=['Product'], values=['Quantity', 'Price'],
              aggfunc={'Quantity': len, 'Price': np.sum}, fill_value=0)

Price                              Quantity  \
Product                     CPU Maintenance Monitor Software      CPU   
Manager       Status                                                    
Debra Henley  won         65000           0       0        0        1   
              pending     40000       10000       0        0        1   
              presented   30000           0       0    20000        1   
              declined    70000           0       0        0        2   
Fred Anderson won        165000        7000       0        0        2   
              pending         0        5000       0        0        0   
              presented   30000           0    5000    10000        1   
              declined    65000           0       0        0        1   

                                                      
Product                 Maintenance Monitor Software  
Manager       Status                                  
Debra Henley  won                 0       0        0  
              pending             2       0        0  
              presented           0       0        2  
              declined            0       0        0  
Fred Anderson won                 1       0        0  
              pending             1       0        0  
              presented           0       1        1  
              declined            0       0        0

In [44]:
table = pd.pivot_table(df, index=['Manager', 'Status'], columns=['Product'], values=['Quantity', 'Price'], 
                       aggfunc={'Quantity': len, 'Price':[np.sum, np.mean]}, fill_value=0)
table

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  won        65000           0       0        0   65000   
              pending    40000        5000       0        0   40000   
              presented  30000           0       0    10000   30000   
              declined   35000           0       0        0   70000   
Fred Anderson won        82500        7000       0        0  165000   
              pending        0        5000       0        0       0   
              presented  30000           0    5000    10000   30000   
              declined   65000           0       0        0   65000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  won                 0       0        0        1           0   
              pending         10000       0        0        1           2   
              presented           0       0    20000        1           0   
              declined            0       0        0        2           0   
Fred Anderson won              7000       0        0        2           1   
              pending          5000       0        0        0           1   
              presented           0    5000    10000        1           0   
              declined            0       0        0        1           0   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  won             0        0  
              pending         0        0  
              presented       0        2  
              declined        0        0  
Fred Anderson won             0        0  
              pending         0        0  
              presented       1        1  
              declined        0        0

> 产生了Table之后，我们就可以通过 **`query`** 来进行查找, 查询的是各个Lable！查询的条件整体是个**字符串**！！

In [47]:
table.query('Manager == ["Debra Henley"]')

Price                                                  \
                         mean                                 sum               
Product                   CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                             
Debra Henley won        65000           0       0        0  65000           0   
             pending    40000        5000       0        0  40000       10000   
             presented  30000           0       0    10000  30000           0   
             declined   35000           0       0        0  70000           0   

                                        Quantity                               
                                             len                               
Product                Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                            
Debra Henley won             0        0        1           0       0        0  
             pending         0        0        1           2       0        0  
             presented       0    20000        1           0       0        2  
             declined        0        0        2           0       0        0

In [48]:
table.query('Status == ["pending", "won"]')

Price                                                   \
                        mean                                  sum               
Product                  CPU Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                            
Debra Henley  won      65000           0       0        0   65000           0   
              pending  40000        5000       0        0   40000       10000   
Fred Anderson won      82500        7000       0        0  165000        7000   
              pending      0        5000       0        0       0        5000   

                                       Quantity                               
                                            len                               
Product               Monitor Software      CPU Maintenance Monitor Software  
Manager       Status                                                          
Debra Henley  won           0        0        1           0       0        0  
              pending       0        0        1           2       0        0  
Fred Anderson won           0        0        2           1       0        0  
              pending       0        0        0           1       0        0